In [10]:
import cv2

In [64]:
image = cv2.imread('data/00918.png')

In [66]:
plate_cascade = cv2.CascadeClassifier('model/haarcascade_russian_plate_number.xml')

In [68]:
plates = plate_cascade.detectMultiScale(image, 1.1, 1)

In [70]:
plates

()

In [72]:
for (x, y, w, h) in plates:
    cropped_plate = image[y:y+h, x:x+w]
    #cv2.imshow("Cropped Plate", cropped_plate)
    cv2.imwrite("2.jpg", cropped_plate)

In [11]:
cv2.waitKey(0)
cv2.destroyAllWindows()

In [66]:
import cv2
import numpy as np

# Load the image
image = cv2.imread("data/plate33.jpg")

# Convert to grayscale
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Apply GaussianBlur to reduce noise
blurred = cv2.GaussianBlur(gray, (5, 5), 0)

# Use Canny edge detector
edges = cv2.Canny(blurred, 100, 200)

# Find contours from the edges
contours, _ = cv2.findContours(edges.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Loop through all the contours
for contour in contours:
    # Approximate the contour to a polygon
    epsilon = 0.02 * cv2.arcLength(contour, True)
    approx = cv2.approxPolyDP(contour, epsilon, True)

    # Check if the contour has 4 vertices (which is typical for license plates)
    if len(approx) == 4:
        # Get the bounding box of the contour
        x, y, w, h = cv2.boundingRect(approx)

        # Crop the license plate region
        cropped_plate = image[y:y+h, x:x+w]

        # Optionally, apply a perspective transformation if the plate is rotated
        # For simplicity, this example does not handle the transformation here

        cv2.imshow("Cropped Plate", cropped_plate)
        cv2.imwrite("cropped_plate323.jpg", cropped_plate)

# Display the original image with contours drawn on it
cv2.drawContours(image, contours, -1, (0, 255, 0), 3)
cv2.imshow("Image with Contours", image)

cv2.waitKey(0)
cv2.destroyAllWindows()


In [17]:
import cv2
import numpy as np

# Load YOLO model files
net = cv2.dnn.readNet("model/yolov4.weights", "model/yolov4.cfg")
layer_names = net.getLayerNames()
output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]

# Load image
image = cv2.imread("data/suv.jpg")
height, width, channels = image.shape

# Prepare the image for YOLO (blob)
blob = cv2.dnn.blobFromImage(image, 0.00392, (416, 416), (0, 0, 0), True, crop=False)

# Set the blob as input to the network
net.setInput(blob)

# Run forward pass to get output
outs = net.forward(output_layers)

# Process the output (bounding boxes, class probabilities, etc.)
for out in outs:
    for detection in out:
        scores = detection[5:]
        class_id = np.argmax(scores)
        confidence = scores[class_id]

        # Filter based on confidence
        if confidence > 0.5:  # You can adjust this threshold
            center_x = int(detection[0] * width)
            center_y = int(detection[1] * height)
            w = int(detection[2] * width)
            h = int(detection[3] * height)

            # Draw bounding box around detected license plate
            cv2.rectangle(image, (center_x - w // 2, center_y - h // 2), 
                          (center_x + w // 2, center_y + h // 2), (0, 255, 0), 2)

# Show the image with detected license plate
cv2.imshow("Detected License Plate", image)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [100]:
import cv2
import numpy as np
import os

def detect_and_crop_license_plate(image_path, output_path="cropped_plate_grok3.jpg"):
    # Read the image
    image = cv2.imread(image_path)
    if image is None:
        raise ValueError("Could not load the image")

    # Convert to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Apply Gaussian blur to reduce noise
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    
    # Edge detection
    edges = cv2.Canny(blurred, 50, 150)
    
    # Find contours
    contours, _ = cv2.findContours(edges.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # Filter contours to find potential license plates
    plate_contour = None
    max_area = 0
    
    for contour in contours:
        # Approximate the contour to a polygon
        peri = cv2.arcLength(contour, True)
        approx = cv2.approxPolyDP(contour, 0.02 * peri, True)
        
        # Check if the contour has 4 sides (rectangle) and reasonable area
        area = cv2.contourArea(contour)
        if len(approx) == 4 and 500 < area < 50000:  # Adjust area range as needed
            # Check aspect ratio (typical license plate is wider than tall)
            x, y, w, h = cv2.boundingRect(contour)
            aspect_ratio = float(w) / h
            if 1.5 < aspect_ratio < 5.0:  # Typical license plate aspect ratio
                if area > max_area:
                    max_area = area
                    plate_contour = approx
    
    if plate_contour is None:
        raise ValueError("No license plate detected in the image")
    
    # Get bounding rectangle coordinates
    x, y, w, h = cv2.boundingRect(plate_contour)
    
    # Add padding to ensure full plate is captured
    padding = 10
    x = max(0, x - padding)
    y = max(0, y - padding)
    w = min(image.shape[1] - x, w + 2 * padding)
    h = min(image.shape[0] - y, h + 2 * padding)
    
    # Crop the license plate
    plate_image = image[y:y+h, x:x+w]
    
    # Optional: Apply perspective transform if plate is skewed
    pts = plate_contour.reshape(4, 2)
    rect = order_points(pts)
    max_width = max(int(np.linalg.norm(rect[0] - rect[1])), int(np.linalg.norm(rect[2] - rect[3])))
    max_height = max(int(np.linalg.norm(rect[0] - rect[3])), int(np.linalg.norm(rect[1] - rect[2])))
    
    dst = np.array([
        [0, 0],
        [max_width - 1, 0],
        [max_width - 1, max_height - 1],
        [0, max_height - 1]], dtype="float32")
    
    # Compute perspective transform
    M = cv2.getPerspectiveTransform(rect, dst)
    warped = cv2.warpPerspective(image, M, (max_width, max_height))
    
    # Save both the simple crop and perspective-corrected version
    cv2.imwrite(output_path, warped)
    cv2.imwrite("simple_crop_" + output_path, plate_image)
    
    return True

def order_points(pts):
    # Order points in a rectangle (top-left, top-right, bottom-right, bottom-left)
    rect = np.zeros((4, 2), dtype="float32")
    
    # Sum and difference of x and y coordinates
    s = pts.sum(axis=1)
    diff = np.diff(pts, axis=1)
    
    rect[0] = pts[np.argmin(s)]  # Top-left (min sum)
    rect[2] = pts[np.argmax(s)]  # Bottom-right (max sum)
    rect[1] = pts[np.argmin(diff)]  # Top-right (min diff)
    rect[3] = pts[np.argmax(diff)]  # Bottom-left (max diff)
    
    return rect

# Example usage
if __name__ == "__main__":
    try:
        # Replace with your image path
        input_image = "data/suv.jpg"
        output_image = "cropped_plate_grok5.jpg"
        
        success = detect_and_crop_license_plate(input_image, output_image)
        if success:
            print(f"License plate successfully cropped and saved as {output_image}")
            print(f"Simple crop saved as simple_crop_{output_image}")
    except Exception as e:
        print(f"Error: {str(e)}")


Error: No license plate detected in the image


In [31]:
import cv2
import numpy as np
import os

def load_yolo_model(weights_path, config_path, names_path):
    net = cv2.dnn.readNet(weights_path, config_path)
    with open(names_path, "r") as f:
        classes = [line.strip() for line in f.readlines()]
    layer_names = net.getLayerNames()
    output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]
    return net, classes, output_layers

def detect_and_crop_license_plate(image_path, weights_path, config_path, names_path, output_path="cropped_plate.jpg", confidence_threshold=0.5):
    # Load the image
    image = cv2.imread(image_path)
    if image is None:
        raise ValueError("Could not load the image")
    
    height, width = image.shape[:2]
    print(f"Image dimensions: {width}x{height}")
    
    # Load YOLO model
    net, classes, output_layers = load_yolo_model(weights_path, config_path, names_path)
    print(f"Classes: {classes}")
    
    # Prepare image for YOLO
    blob = cv2.dnn.blobFromImage(image, 1/255.0, (416, 416), swapRB=True, crop=False)
    net.setInput(blob)
    
    # Perform detection
    outputs = net.forward(output_layers)
    
    # Process detections
    boxes = []
    confidences = []
    class_ids = []
    
    for output in outputs:
        for detection in output:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            
            if confidence > confidence_threshold:
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)
                
                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)
                print(f"Detected: Class={classes[class_id]}, Confidence={confidence}, Box=[{x}, {y}, {w}, {h}]")
    
    if not boxes:
        raise ValueError("No objects detected above confidence threshold")
    
    # Apply Non-Max Suppression
    indices = cv2.dnn.NMSBoxes(boxes, confidences, confidence_threshold, 0.4)
    print(f"NMS indices: {indices}")
    
    if len(indices) == 0:
        raise ValueError("No license plate detected after NMS")
    
    # Get the best detection
    idx = indices[0]  # Take the first detection
    box = boxes[idx]
    x, y, w, h = box
    print(f"Selected box: [{x}, {y}, {w}, {h}]")
    
    # Add padding and ensure coordinates are within image bounds
    padding = 10
    x = max(0, x - padding)
    y = max(0, y - padding)
    w = min(width - x, w + 2 * padding)
    h = min(height - y, h + 2 * padding)
    print(f"Adjusted box with padding: [{x}, {y}, {w}, {h}]")
    
    # Crop the license plate
    plate_image = image[y:y+h, x:x+w]
    if plate_image.size == 0:
        raise ValueError("Cropped image is empty - invalid coordinates")
    
    # Save the cropped image
    cv2.imwrite(output_path, plate_image)
    print(f"Cropped image shape: {plate_image.shape}")
    
    return True

# Example usage
if __name__ == "__main__":
    try:
        input_image = "data/suv.jpg"
        output_image = "cropped_plate123.jpg"
        weights_path = "model/yolov4.weights"
        config_path = "model/yolov4.cfg"
        names_path = "model/coco.names"
        
        success = detect_and_crop_license_plate(
            input_image,
            weights_path,
            config_path,
            names_path,
            output_image,
            confidence_threshold=0.5
        )
        if success:
            print(f"License plate successfully cropped and saved as {output_image}")
    except Exception as e:
        print(f"Error: {str(e)}")

Image dimensions: 600x433
Classes: ['license_plate', 'license plate']
Error: list index out of range
